In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data

import time, json, datetime
from tqdm import tqdm
import sys, getopt

import time
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
class DeepFM(nn.Module):
    def __init__(self, cate_fea_nuniqs, nume_fea_size=0, emb_size=8,
                 hid_dims=[256, 128], num_classes=1, dropout=[0.2, 0.2]):
        """
        cate_fea_nuniqs: 类别特征的唯一值个数列表，也就是每个类别特征的vocab_size所组成的列表
        nume_fea_size: 数值特征的个数，该模型会考虑到输入全为类别型，即没有数值特征的情况
        """
        super().__init__()
        self.cate_fea_size = len(cate_fea_nuniqs)
        self.nume_fea_size = nume_fea_size

        """FM部分"""
        # 一阶
        if self.nume_fea_size != 0:
            self.fm_1st_order_dense = nn.Linear(self.nume_fea_size, 1)  # 数值特征的一阶表示
        self.fm_1st_order_sparse_emb = nn.ModuleList([
            nn.Embedding(voc_size, 1) for voc_size in cate_fea_nuniqs])  # 类别特征的一阶表示

        # 二阶
        self.fm_2nd_order_sparse_emb = nn.ModuleList([
            nn.Embedding(voc_size, emb_size) for voc_size in cate_fea_nuniqs])  # 类别特征的二阶表示

        """DNN部分"""
        self.all_dims = [self.cate_fea_size * emb_size] + hid_dims
        self.dense_linear = nn.Linear(self.nume_fea_size, self.cate_fea_size * emb_size)  # 数值特征的维度变换到FM输出维度一致
        self.relu = nn.ReLU()
        # for DNN
        for i in range(1, len(self.all_dims)):
            setattr(self, 'linear_' + str(i), nn.Linear(self.all_dims[i - 1], self.all_dims[i]))
            setattr(self, 'batchNorm_' + str(i), nn.BatchNorm1d(self.all_dims[i]))
            setattr(self, 'activation_' + str(i), nn.ReLU())
            setattr(self, 'dropout_' + str(i), nn.Dropout(dropout[i - 1]))
        # for output
        self.dnn_linear = nn.Linear(hid_dims[-1], num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X_sparse, X_dense=None):
        """
        X_sparse: 类别型特征输入  [bs, cate_fea_size]
        X_dense: 数值型特征输入（可能没有）  [bs, dense_fea_size]
        """

        """FM 一阶部分"""
        fm_1st_sparse_res = [emb(X_sparse[:, i].unsqueeze(1)).view(-1, 1)
                             for i, emb in enumerate(self.fm_1st_order_sparse_emb)]
        fm_1st_sparse_res = torch.cat(fm_1st_sparse_res, dim=1)  # [bs, cate_fea_size]
        fm_1st_sparse_res = torch.sum(fm_1st_sparse_res, 1, keepdim=True)  # [bs, 1]

        if X_dense is not None:
            fm_1st_dense_res = self.fm_1st_order_dense(X_dense)
            fm_1st_part = fm_1st_sparse_res + fm_1st_dense_res
        else:
            fm_1st_part = fm_1st_sparse_res  # [bs, 1]

        """FM 二阶部分"""
        fm_2nd_order_res = [emb(X_sparse[:, i].unsqueeze(1)) for i, emb in enumerate(self.fm_2nd_order_sparse_emb)]
        fm_2nd_concat_1d = torch.cat(fm_2nd_order_res, dim=1)  # [bs, n, emb_size]  n为类别型特征个数(cate_fea_size)

        # 先求和再平方
        sum_embed = torch.sum(fm_2nd_concat_1d, 1)  # [bs, emb_size]
        square_sum_embed = sum_embed * sum_embed  # [bs, emb_size]
        # 先平方再求和
        square_embed = fm_2nd_concat_1d * fm_2nd_concat_1d  # [bs, n, emb_size]
        sum_square_embed = torch.sum(square_embed, 1)  # [bs, emb_size]
        # 相减除以2
        sub = square_sum_embed - sum_square_embed
        sub = sub * 0.5  # [bs, emb_size]

        fm_2nd_part = torch.sum(sub, 1, keepdim=True)  # [bs, 1]

        """DNN部分"""
        dnn_out = torch.flatten(fm_2nd_concat_1d, 1)  # [bs, n * emb_size]

        if X_dense is not None:
            dense_out = self.relu(self.dense_linear(X_dense))  # [bs, n * emb_size]
            dnn_out = dnn_out + dense_out  # [bs, n * emb_size]

        for i in range(1, len(self.all_dims)):
            dnn_out = getattr(self, 'linear_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'batchNorm_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'activation_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'dropout_' + str(i))(dnn_out)

        dnn_out = self.dnn_linear(dnn_out)  # [bs, 1]
        out = fm_1st_part + fm_2nd_part + dnn_out  # [bs, 1]
        out = self.sigmoid(out)
        return out

In [4]:
def train_and_eval(model, train_loader, valid_loader, epochs, optimizer, loss_fcn, scheduler, device,ii):
    import time
    best_auc=0
    for _ in range(epochs):
        """训练部分"""
        model.train()
        print("Current lr : {}".format(optimizer.state_dict()['param_groups'][0]['lr']))
        write_log('Epoch: {}:{}'.format(ii,_))
        train_loss_sum = 0.0
        start_time = time.time()
        for idx, x in enumerate(train_loader):
            cate_fea, nume_fea, label = x[0], x[1], x[2]
            cate_fea, nume_fea, label = cate_fea.to(device), nume_fea.to(device), label.float().to(device)
            pred = model(cate_fea, nume_fea).view(-1)
            loss = loss_fcn(pred, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_sum += loss.cpu().item()
            if (idx + 1) % 50 == 0 or (idx + 1) == len(train_loader):
                write_log("Epoch {:04d} | Step {:04d} / {} | Loss {:.4f} | Time {:.4f}".format(
                    _ + 1, idx + 1, len(train_loader), train_loss_sum / (idx + 1), time.time() - start_time))
        scheduler.step()
        """推断部分"""
        model.eval()
        with torch.no_grad():
            valid_labels, valid_preds = [], []
            for idx, x in tqdm(enumerate(valid_loader)):
                cate_fea, nume_fea, label = x[0], x[1], x[2]
                cate_fea, nume_fea = cate_fea.to(device), nume_fea.to(device)
                pred = model(cate_fea, nume_fea).reshape(-1).data.cpu().numpy().tolist()
                valid_preds.extend(pred)
                valid_labels.extend(label.cpu().numpy().tolist())
        cur_auc = roc_auc_score(valid_labels, valid_preds)
        if cur_auc > best_auc:
            best_auc = cur_auc
            import time
            end=time.time()
            torch.save(model.state_dict(), "../../data/wj/deepfm_best/fen10 4.0 shan/deepfm_best10fen_"+str(round(best_auc,4))+"_"+str(ii)+"_"+str(_)+"_"+str(time.strftime('%m_%d_%H_%M_%S'))+".pth")
        write_log('Current AUC: %.6f, Best AUC: %.6f\n' % (cur_auc, best_auc))




# 定义日志（data文件夹下，同级目录新建一个data文件夹）
def write_log(w):
    file_name = '../../data/wj/deepfm_best/fen10 4.0 shan/' + datetime.date.today().strftime('%m%d') + "_{}.log".format("deepfm_10fen_user")
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}".format(t0, w)
    print(info)
    with open(file_name, 'a') as f:
        f.write(info + '\n')
        
        

In [6]:
doc_feat = pd.read_pickle('../../data/wj/doc.pkl')
user_feat = pd.read_pickle('../../data/wj/user.pkl')
df_test_user_doc=pd.read_pickle('../../data/wj/df_test_user_doc_64_new.pkl')

sparse_features = ['userid', 'docid', 'network', 'hour', 'device', 'os', 'province',
                   'city', 'age', 'gender', 'category1st', 'category2nd',
                   'pub_date', 'keyword0', 'keyword1', 'keyword2', 'keyword3', 'keyword4']

# nono:'userid_duration_mean','userid_click_mean','docid_click_mean'

dense_features = ['refresh', 'picnum',
                  'userid_click_count' ,'userid_duration_mean' ,'userid_picnum_mean',
                   'docid_click_count','docid_duration_mean','docid_picnum_mean',
                    'category1st_click_mean','category1st_click_count','category1st_duration_mean','category1st_picnum_mean',
                    'category2nd_click_mean','category2nd_click_count','category2nd_duration_mean','category2nd_picnum_mean',
                    'keyword0_click_mean','keyword0_click_count','keyword0_duration_mean','keyword0_picnum_mean',
                 'network_click_mean', 'network_click_count', 'network_duration_mean', 
                  'refresh_click_mean', 'refresh_click_count', 'refresh_duration_mean',
                  'device_click_mean', 'device_click_count', 'device_duration_mean', 
                  'os_click_mean', 'os_click_count', 'os_duration_mean', 
                  'province_click_mean', 'province_click_count', 'province_duration_mean', 
                  'city_click_mean', 'city_click_count', 'city_duration_mean', 
                  'age_click_mean', 'age_click_count', 'age_duration_mean', 
                  'gender_click_mean', 'gender_click_count', 'gender_duration_mean'
                 ]

cate_fea_nuniqs = []
cate_fea_nuniqs.append(user_feat['userid'].nunique() + 1)
cate_fea_nuniqs.append(doc_feat['docid'].nunique() + 1)
cate_fea_nuniqs.append(6)  # network
cate_fea_nuniqs.append(13)  # hour
cate_fea_nuniqs.append(user_feat['device'].nunique())
cate_fea_nuniqs.append(user_feat['os'].nunique())
cate_fea_nuniqs.append(user_feat['province'].nunique())
cate_fea_nuniqs.append(user_feat['city'].nunique())
cate_fea_nuniqs.append(user_feat['age'].nunique())
cate_fea_nuniqs.append(user_feat['gender'].nunique())
cate_fea_nuniqs.append(doc_feat['category1st'].nunique())
cate_fea_nuniqs.append(doc_feat['category2nd'].nunique())
cate_fea_nuniqs.append(doc_feat['pub_date'].nunique())
keyword_nunique = max(doc_feat['keyword0'].max(), doc_feat['keyword1'].max(), doc_feat['keyword2'].max()
                      , doc_feat['keyword3'].max(), doc_feat['keyword4'].max()) + 1
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)

device = torch.device('cuda:2') if torch.cuda.is_available() else torch.device('cpu')
loss_fcn = nn.BCELoss()
loss_fcn = loss_fcn.to(device)

In [ ]:
import pandas as pd


len_=81671133
ii=-1
for i in range(0,len_,round(len_/10)):
    ii+=1
    right=i+round(len_/10)
    if right>=len_:
        break
    if i==0:
        continue
    print(i,right)
    df_train_user_doc=pd.read_pickle('../../data/wj/df_train_user_doc_0_1_'+str(i)+'_'+str(right)+'_64.pkl')

    train, valid = train_test_split(df_train_user_doc, test_size=0.25, random_state=2021)
#         train_loader，valid_loader
    train_dataset = Data.TensorDataset(torch.LongTensor(train[sparse_features].values),
                                       torch.FloatTensor(train[dense_features].values),
                                       torch.FloatTensor(train['click'].values), )
    train_loader = Data.DataLoader(dataset=train_dataset, batch_size=8192, shuffle=True)
    valid_dataset = Data.TensorDataset(torch.LongTensor(valid[sparse_features].values),
                                       torch.FloatTensor(valid[dense_features].values),
                                       torch.FloatTensor(valid['click'].values))
    valid_loader = Data.DataLoader(dataset=valid_dataset, batch_size=8192, shuffle=False)
    
#         train
    print('train')
    model = DeepFM(cate_fea_nuniqs, nume_fea_size=len(dense_features))
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
    epoch_=15
    train_and_eval(model, train_loader, valid_loader, epoch_, optimizer, loss_fcn, scheduler, device,ii)

8167113 16334226
train
Current lr : 0.005
12:31:52 : Epoch: 1:0
12:32:06 : Epoch 0001 | Step 0050 / 748 | Loss 11.9426 | Time 14.0571
12:32:20 : Epoch 0001 | Step 0100 / 748 | Loss 8.6573 | Time 27.3105
12:32:32 : Epoch 0001 | Step 0150 / 748 | Loss 6.6071 | Time 39.8790
12:32:44 : Epoch 0001 | Step 0200 / 748 | Loss 5.2230 | Time 52.0843
12:32:57 : Epoch 0001 | Step 0250 / 748 | Loss 4.3248 | Time 64.3849
12:33:09 : Epoch 0001 | Step 0300 / 748 | Loss 3.7008 | Time 76.5253
12:33:21 : Epoch 0001 | Step 0350 / 748 | Loss 3.2433 | Time 89.1620
12:33:34 : Epoch 0001 | Step 0400 / 748 | Loss 2.8945 | Time 101.3049
12:33:47 : Epoch 0001 | Step 0450 / 748 | Loss 2.6206 | Time 114.4486
12:33:59 : Epoch 0001 | Step 0500 / 748 | Loss 2.3998 | Time 126.9538
12:34:12 : Epoch 0001 | Step 0550 / 748 | Loss 2.2181 | Time 139.7606
12:34:26 : Epoch 0001 | Step 0600 / 748 | Loss 2.0660 | Time 153.2284
12:34:39 : Epoch 0001 | Step 0650 / 748 | Loss 1.9367 | Time 166.5766
12:34:52 : Epoch 0001 | Step 070

250it [00:50,  4.97it/s]


12:35:56 : Current AUC: 0.708290, Best AUC: 0.708290

Current lr : 0.004
12:35:56 : Epoch: 1:1
12:36:11 : Epoch 0002 | Step 0050 / 748 | Loss 0.3775 | Time 15.0792
12:36:25 : Epoch 0002 | Step 0100 / 748 | Loss 0.3776 | Time 29.6201
12:36:39 : Epoch 0002 | Step 0150 / 748 | Loss 0.3773 | Time 43.6336
12:36:53 : Epoch 0002 | Step 0200 / 748 | Loss 0.3766 | Time 57.1690
12:37:06 : Epoch 0002 | Step 0250 / 748 | Loss 0.3764 | Time 70.7124
12:37:21 : Epoch 0002 | Step 0300 / 748 | Loss 0.3757 | Time 84.8915
12:37:34 : Epoch 0002 | Step 0350 / 748 | Loss 0.3753 | Time 98.7718
12:37:49 : Epoch 0002 | Step 0400 / 748 | Loss 0.3749 | Time 113.0783
12:38:03 : Epoch 0002 | Step 0450 / 748 | Loss 0.3747 | Time 127.2295
12:38:17 : Epoch 0002 | Step 0500 / 748 | Loss 0.3743 | Time 141.2225
12:38:31 : Epoch 0002 | Step 0550 / 748 | Loss 0.3739 | Time 155.4451
12:38:46 : Epoch 0002 | Step 0600 / 748 | Loss 0.3736 | Time 170.5020
12:39:00 : Epoch 0002 | Step 0650 / 748 | Loss 0.3734 | Time 184.4966
12

250it [00:57,  4.37it/s]


12:40:26 : Current AUC: 0.737169, Best AUC: 0.737169

Current lr : 0.0032
12:40:26 : Epoch: 1:2
12:40:43 : Epoch 0003 | Step 0050 / 748 | Loss 0.3697 | Time 16.0673
12:40:58 : Epoch 0003 | Step 0100 / 748 | Loss 0.3688 | Time 31.6180
12:41:14 : Epoch 0003 | Step 0150 / 748 | Loss 0.3681 | Time 47.2817
12:41:29 : Epoch 0003 | Step 0200 / 748 | Loss 0.3676 | Time 62.8511
12:41:44 : Epoch 0003 | Step 0250 / 748 | Loss 0.3676 | Time 77.6185
12:41:58 : Epoch 0003 | Step 0300 / 748 | Loss 0.3676 | Time 91.9438
12:42:13 : Epoch 0003 | Step 0350 / 748 | Loss 0.3677 | Time 106.8874
12:42:27 : Epoch 0003 | Step 0400 / 748 | Loss 0.3676 | Time 120.2432
12:42:40 : Epoch 0003 | Step 0450 / 748 | Loss 0.3677 | Time 133.8507
12:42:55 : Epoch 0003 | Step 0500 / 748 | Loss 0.3678 | Time 148.5246
12:43:10 : Epoch 0003 | Step 0550 / 748 | Loss 0.3676 | Time 163.6286
12:43:25 : Epoch 0003 | Step 0600 / 748 | Loss 0.3675 | Time 178.4009
12:43:40 : Epoch 0003 | Step 0650 / 748 | Loss 0.3675 | Time 193.0377


...

14:38:15 : Current AUC: 0.756212, Best AUC: 0.757402

Current lr : 0.0002199023255552002
14:38:15 : Epoch: 2:14
14:38:30 : Epoch 0015 | Step 0050 / 748 | Loss 0.3266 | Time 15.0879
14:38:44 : Epoch 0015 | Step 0100 / 748 | Loss 0.3268 | Time 29.0430
14:38:57 : Epoch 0015 | Step 0150 / 748 | Loss 0.3283 | Time 42.4316
14:39:11 : Epoch 0015 | Step 0200 / 748 | Loss 0.3295 | Time 56.3040
14:39:24 : Epoch 0015 | Step 0250 / 748 | Loss 0.3310 | Time 69.6884
14:39:38 : Epoch 0015 | Step 0300 / 748 | Loss 0.3325 | Time 83.1520
14:39:52 : Epoch 0015 | Step 0350 / 748 | Loss 0.3339 | Time 97.8315
14:40:07 : Epoch 0015 | Step 0400 / 748 | Loss 0.3351 | Time 112.0857
14:40:22 : Epoch 0015 | Step 0450 / 748 | Loss 0.3362 | Time 127.0451
14:41:19 : Epoch 0015 | Step 0650 / 748 | Loss 0.3394 | Time 184.4528
14:41:33 : Epoch 0015 | Step 0700 / 748 | Loss 0.3401 | Time 198.3655
14:41:46 : Epoch 0015 | Step 0748 / 748 | Loss 0.3406 | Time 211.1540


250it [00:54,  4.58it/s]


14:42:42 : Current AUC: 0.746264, Best AUC: 0.757402

24501339 32668452
train
Current lr : 0.005
14:43:10 : Epoch: 3:0
14:43:25 : Epoch 0001 | Step 0050 / 748 | Loss 14.9631 | Time 14.7449
14:43:39 : Epoch 0001 | Step 0100 / 748 | Loss 10.3543 | Time 28.3206
14:43:51 : Epoch 0001 | Step 0150 / 748 | Loss 7.9041 | Time 41.2518
14:44:04 : Epoch 0001 | Step 0200 / 748 | Loss 6.2529 | Time 54.1194
14:44:18 : Epoch 0001 | Step 0250 / 748 | Loss 5.1646 | Time 67.4095
14:44:30 : Epoch 0001 | Step 0300 / 748 | Loss 4.4075 | Time 79.8331
14:44:43 : Epoch 0001 | Step 0350 / 748 | Loss 3.8536 | Time 92.8932
14:44:56 : Epoch 0001 | Step 0400 / 748 | Loss 3.4318 | Time 105.4021
14:45:08 : Epoch 0001 | Step 0450 / 748 | Loss 3.1004 | Time 118.2091
14:45:22 : Epoch 0001 | Step 0500 / 748 | Loss 2.8328 | Time 131.5229
14:45:35 : Epoch 0001 | Step 0550 / 748 | Loss 2.6127 | Time 144.5081
14:45:47 : Epoch 0001 | Step 0600 / 748 | Loss 2.4284 | Time 157.1862
14:46:01 : Epoch 0001 | Step 0650 / 748 | Loss

250it [00:51,  4.84it/s]


14:47:19 : Current AUC: 0.693694, Best AUC: 0.693694

Current lr : 0.004
14:47:19 : Epoch: 3:1
14:47:34 : Epoch 0002 | Step 0050 / 748 | Loss 0.3847 | Time 15.1914
14:47:47 : Epoch 0002 | Step 0100 / 748 | Loss 0.3839 | Time 28.3889


In [38]:
torch.save(model.state_dict(), "../../data/wj/deepfm_best_"+str(0.7791)+"_"+str(1)+"_"+str(9)+".pth")

In [22]:
# df_train_user_doc.to_pickle('../../data/wj/df_train_user_doc_0.1.pkl')

In [5]:
# model = DeepFM(cate_fea_nuniqs, nume_fea_size=len(dense_features))
# model.load_state_dict(torch.load('../../data/wj/deepfm_best_0.7739082162724398_1639060823.4660978.pth'))
# model.to(device)
# epoch = 5
# train_and_eval(model, train_loader, valid_loader, epoch, optimizer, loss_fcn, scheduler, device)